In [9]:
# 2026/1/30
# zhangzhong
# Reinforcement Learning with Simple Code
# tutorial from ChatGPT


# roblem: A Robot in a 1D World

Imagine a world like this:
[ Start ] --- --- --- --- [ Goal ]
   0       1   2   3   4

Robot starts at position 0

Goal is at position 4

Robot can do 2 actions:
	•	LEFT
	•	RIGHT

Rewards:
	•	+10 if reach the goal
	•	-1 for every step (to encourage faster solutions)

In [10]:
# Step 1 — Environment Code
# # Now we have an environment where the agent can interact.    

class SimpleWorld:
    def __init__(self):
        self.goal = 4
        self.state = 0   # start position

    def step(self, action):
        if action == "RIGHT":
            self.state += 1
        elif action == "LEFT":
            self.state -= 1

        # keep inside bounds
        self.state = max(0, min(self.state, self.goal))

        # reward
        if self.state == self.goal:
            return self.state, 10, True   # done
        else:
            return self.state, -1, False  # not done
        
    

In [11]:
# Step 2
# This agent just moves randomly.
# Sometimes it reaches the goal, often not efficiently.

import random

env = SimpleWorld()

done = False
while not done:
    action = random.choice(["LEFT", "RIGHT"])
    state, reward, done = env.step(action)

    print("Action:", action, "State:", state, "Reward:", reward)


Action: LEFT State: 0 Reward: -1
Action: LEFT State: 0 Reward: -1
Action: RIGHT State: 1 Reward: -1
Action: RIGHT State: 2 Reward: -1
Action: RIGHT State: 3 Reward: -1
Action: LEFT State: 2 Reward: -1
Action: LEFT State: 1 Reward: -1
Action: RIGHT State: 2 Reward: -1
Action: RIGHT State: 3 Reward: -1
Action: RIGHT State: 4 Reward: 10


In [12]:
# Step 3: The Agent Needs Memory (Q-values)
# The agent should learn:“Going RIGHT is usually good”
# So we store values: Q[state][action] = how good it is

# The Q-learning formula
# Q(s,a) = Q(s,a) + \alpha \Big[r + \gamma \max Q(s') - Q(s,a)\Big]
# \text{New estimate} = \text{Old estimate} + \text{Correction}
# New estimate = Old estimate + Correction
# Example: 
# - Old estimate: you have some experience
# - CorrectionL you encouter some new events
# - New estimate: you got new experience

# Q(s,a): How good is it to take action a in state s?”
# Example:
# - State: position in maze
# - Action: move right
# - Q-value: expected future reward
# Q-values measure [future usefulness], not just immediate reward.
# Q-value means The expected total future reward if you take action a in state s.
# Q-value is just a real nubmer:
#   - positive means good action
#   - negative means bad action

## How do we calculate Q(s,a)?
# 1. start with guess or 0s.
# 2. then the agent interact with the env and get experience
#   Then the agent interacts:
#   Example experience:
# 	•	State: 0
# 	•	Action: RIGHT
# 	•	Reward: -1
# 	•	Next state: 1
#   Then update:
#   Q(0,\text{RIGHT}) = Q(0,\text{RIGHT}) + \alpha[\dots]

## Value Function, V(s)
# How good is state s, assuming the agent acts optimally?
# 在状态s下，假设执行所有可选动作a，能获得的最大的reward
# V(s) = \max_a Q(s,a)
# Example:
# - Q(s,\text{LEFT}) = 3
# - Q(s,\text{RIGHT}) = 7
# V(s) = \max(3,7)=7

## How do we represent Q-values?
# 1. Small world -> Table, Q[s][a]
# 2. Large world -> Neural Network (Deep Q Learning), Q(s, a) = Net(s, a)

## let's explain the whole formula
# so we start with the current(old) experience(estimation)

# \alpha is learning rate: Controls how fast we learn from new events

# r: reward, the reward of take action a at state s. This is the agent’s feedback.
#   - +10 if goal reached
#   - -1 for step cost

# \gamma: discount factor
#   Controls importance of [future rewards]. what is future reward?
#   	\gamma = 0: only care about immediate reward
#       \gamma = 0.99: care a lot about future reward
# example: Would you rather get $10 today or $10 next week?


## Target！
# Target = r+\gamma\max_{a'}Q(s',a')
# 

# Q-Learning
下面是一份把我们刚才围绕 Q-value / Q-learning 更新公式 的讨论整理成的详细笔记（包含你提出的所有疑问 + 对应回答），你可以直接复制到笔记软件里。

⸻

强化学习 Q-value / Q-learning 更新公式笔记（含疑问整理）

0. 背景：我们在讨论什么？

我们讨论的是 Q-learning 中最核心的更新规则（Temporal Difference / TD learning）：

Q(s_t,a_t)\leftarrow Q(s_t,a_t)+\alpha\Big[r_{t+1}+\gamma\max_{a'}Q(s_{t+1},a')-Q(s_t,a_t)\Big]

这条规则告诉我们：执行一步交互后，如何更新 “在状态 s 采取动作 a” 的价值估计。

⸻

1) 疑问：公式里到底哪个是“下一个状态”？我看不出来状态转移

你指出：“从一个状态经过一个 action 应该到达另一个状态，但公式里看不出来。”

✅ 澄清：一次交互的完整形式

一次交互产生四元组：

(s_t,\ a_t,\ r_{t+1},\ s_{t+1})
	•	当前状态：s_t
	•	当前动作：a_t
	•	执行动作后得到的奖励：r_{t+1}
	•	执行动作后到达的下一个状态：s_{t+1}

✅ 正确、严格的 Q-learning 更新式

你说得对：未来部分必须写出动作维度，否则会歧义：

Q(s_t,a_t)\leftarrow Q(s_t,a_t)
+\alpha\Big[r_{t+1}+\gamma\max_{a'} Q(s_{t+1},a')-Q(s_t,a_t)\Big]

其中 \max_{a'} Q(s_{t+1},a') 表示：在 下一个状态 s_{t+1} 下，尝试所有动作 a'，选 Q 值最大的那个。

之前把它简写成 \max Q(s') 的确不严谨，会让人看不出来在对哪个变量取 max。

⸻

2) 疑问：Q(s,a) 怎么计算？Q(s) 又是什么？Q 值是不是实数？

✅ Q(s,a) 是什么？

Q(s,a)
含义：

在状态 s 执行动作 a 后，未来能获得的 折扣累计回报 的期望（预测值）。

✅ Q 值是不是实数？

是的，Q-value 就是一个 实数（real number），可以是正、负或小数，比如 5.6、-3.2、0 等。

✅ Q(s) 是什么？

一般我们不用 Q(s)（因为 Q 通常需要 action），更标准的是 状态价值函数：

V(s)

定义：

处于状态 s 的价值（假设之后采取最优动作或某个策略）。

二者关系（最常见的一种）：

V(s)=\max_a Q(s,a)

所以：
	•	Q(s,a)：状态+动作 的价值
	•	V(s)：状态本身的价值（取最优动作时）

✅ Q(s,a) 到底怎么得到？

不是直接算出来的，而是从经验中学出来的：
	•	初始化：通常设为 0（或随机小值）
	•	每走一步就用更新公式修正一次
	•	经过很多次交互，Q 会慢慢接近真实的最优 Q 值

⸻

3) 疑问：公式里为什么要减去 Q(s,a)？为什么不每次执行动作就把 reward 加到 Q 里？

你提出了一个非常核心的疑问：

“每次得到 reward（可正可负）就加到 Q-value 里不行吗？为什么要减去旧的 Q(s,a)？”

✅ 关键澄清：Q-value 不是“奖励累加器”

Q(s,a) 是对 长期总回报 的预测，不是“累计奖励的总和”。

如果你每次都做：

Q(s,a)\leftarrow Q(s,a)+r

会出现两个问题：
	1.	Q 会随着次数不断累加，可能越加越大或越小，不收敛
	2.	Q 无法表达“这个动作长期是否值得”（例如每步 -1 但最终 +10 的路径）

✅ 为什么要减去 Q(s,a)？

看更新式的括号部分：

r+\gamma\max_{a'}Q(s',a')-Q(s,a)

这是一个误差项，叫 TD error（时间差分误差）：

新观察到的“更合理目标” 与 旧预测之间的差距。

	•	Q(s,a)：旧预测（旧估计）
	•	r+\gamma\max Q(s',a')：基于最新经验的“新目标”(Target)

更新本质是：

Q \leftarrow Q + \alpha(\text{Target} - Q)

这是一种非常通用的学习结构：预测 → 计算误差 → 修正预测。

✅ 类比帮助理解

你预测考试 80 分（旧 Q），结果你根据新信息觉得应该是 90（Target）：
	•	你不会 80+90
	•	你会把 80 朝 90 调整一点点

⸻

4) 疑问：Target 是不是我们期望的 Q-value？大约等于它应该变成的值？

你总结得非常准确：

“Target 就是我们期望的 Q-value，或者大约等于它应该变成的值。”

✅ 更精确的表达

Target 是对真实最优 Q 值 Q^\*(s,a) 的一个 更好的估计：

\text{Target} = r_{t+1} + \gamma \max_{a'} Q(s_{t+1}, a')

并且学习目标是逐渐让：

Q(s_t,a_t) \approx \text{Target}

每次更新只迈一小步（乘上 \alpha）来避免震荡、保证稳定。

⸻

5) 疑问：这是不是很像动态规划（DP）？

你的直觉完全正确：

Q-learning 非常像动态规划。

✅ 为什么像？

它们都基于 Bellman 递推思想：

当前价值 = 当前收益 + 未来价值

Q-learning 的 Target 形式其实就是 Bellman optimality 的采样版本。

✅ RL 和 DP 的关键区别
	•	DP（动态规划）：通常需要已知环境模型（转移概率 P(s'|s,a)），可以对所有可能 s' 做期望计算
	•	RL（Q-learning）：不知道模型，只能通过采样经验 (s,a,r,s') 来近似更新

一句话：

RL = 动态规划思想 + 采样（model-free）学习

⸻

总结：我们讨论得到的核心认知
	1.	Q(s,a) 是一个实数：表示从 s 做 a 的长期回报预测
	2.	一步交互是：(s_t,a_t,r_{t+1},s_{t+1})
	3.	更新式必须写清楚：\max_{a'}Q(s_{t+1},a')，否则会歧义
	4.	减去 Q(s,a) 是为了计算误差（TD error），让估计逐渐收敛
	5.	Target 是 “当前奖励 + 下一个状态的最优未来价值”，是对真实 Q 的更好估计
	6.	Q-learning 的结构非常像 DP：都基于 Bellman 递推，但 RL 用采样不需要已知模型

⸻

如果你愿意，我还可以把这份笔记再补一个小节：
“Q-learning vs SARSA（为什么一个用 max，一个用实际动作）”，这通常是下一个最自然的疑问。

In [13]:
import random

actions = ["LEFT", "RIGHT"]

# initialize Q-table
Q = {}
for state in range(5):
    Q[state] = {a: 0 for a in actions}

# parameters
alpha = 0.1      # learning rate
gamma = 0.9      # future reward importance
epsilon = 0.2    # exploration probability

env = SimpleWorld()

# training loop
for episode in range(200):

    env.state = 0
    done = False

    while not done:

        state = env.state

        # choose action (explore vs exploit)
        if random.random() < epsilon:
            action = random.choice(actions)
        else:
            # 这里应该就是选择Q最大的action，不过我没太看明白这个写法
            action = max(Q[state], key=Q[state].get)

        # take step
        next_state, reward, done = env.step(action)

        # Q-learning update, 这个就是V
        best_next = max(Q[next_state].values())

        Q[state][action] += alpha * (
            reward + gamma * best_next - Q[state][action]
        )

print("Learned Q-table:")
print(Q)

Learned Q-table:
{0: {'LEFT': 2.697266398169901, 'RIGHT': 4.579997849013276}, 1: {'LEFT': 2.7865865516327895, 'RIGHT': 6.19999957139955}, 2: {'LEFT': 3.7522975146988777, 'RIGHT': 7.999999941377253}, 3: {'LEFT': 5.301321426399191, 'RIGHT': 9.99999999294492}, 4: {'LEFT': 0, 'RIGHT': 0}}
